In [1]:
from helper_funcs.preprocessing import total_timeseries, get_covariates
from helper_funcs.prediction import historical_predictions, display_prediction_part
from helper_funcs.error import error_print
from helper_funcs.inverse import inverse_func

from darts.utils.model_selection import train_test_split
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from darts.models import NLinearModel, TFTModel

import torch

from models import nlinear, tft

RANDOM = 101
INPUT_CHUNK = 15
OUTPUT_CHUNK = 5
TEST_SIZE = 0.2
RETRAIN=True
LAST=False
RESET=False
EXP_MA = 15

checkpoint = ModelCheckpoint(monitor="val_loss")
progress_bar = RichProgressBar()

In [2]:
callbacks=[progress_bar]

In [3]:
timeseries = total_timeseries(market=True, sentiment=True, embeddings=True)
timeseries = StaticCovariatesTransformer().fit_transform(timeseries)

In [4]:
train, val = train_test_split(
    timeseries,
    axis=1,
    test_size=TEST_SIZE,
    input_size=INPUT_CHUNK,
    horizon=OUTPUT_CHUNK,
    vertical_split_type='model-aware'
)

data = [train, val]

target_train, past_train, future_train, target_val, past_val, future_val, target_test, past_test, future_test = get_covariates(
    data, 
    [
        # 'Adj Close',
        # 'High',
        # 'Low',
        # 'Open',
        'Volume',
        # 'Volatility',
        # 'Negative',
        # 'Positive',
        # 'sentiment_score_1',
        # 'sentiment_score_2'
        ],
    embeddings=False
    )

name = 'model_tft(volu_only)'

In [5]:
past_train[0].components

Index(['Volume'], dtype='object', name='component')

In [6]:
scaler_target = Scaler()
scaler_covariates = Scaler()

In [7]:
target_train_scaled = scaler_target.fit_transform(target_train)
target_val_scaled = scaler_target.transform(target_val)
target_test_scaled = scaler_target.transform(target_test)

past_train_scaled = scaler_covariates.fit_transform(past_train)
past_val_scaled = scaler_covariates.transform(past_val)
past_test_scaled = scaler_covariates.transform(past_test)

In [ ]:
# model_nlinear = nlinear.nlinear_emb(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
#     callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
#         future_train, future_val)

In [8]:
model_tft = tft.tft_set(name, INPUT_CHUNK, OUTPUT_CHUNK, RANDOM, \
    callbacks, target_train_scaled, target_val_scaled, past_train_scaled, past_val_scaled, \
        future_train, future_val)

ValueError: Could not find base model save file `_model.pth.tar` in /mnt/windows/wd_blue/HSE_Masters/darts_logs/model_tft(volu_only).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                              ┃ Type                             ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ train_metrics                     │ MetricCollection                 │      0 │
│ 1  │ val_metrics                       │ MetricCollection                 │      0 │
│ 2  │ input_embeddings                  │ _MultiEmbedding                  │      0 │
│ 3  │ static_covariates_vsn             │ _VariableSelectionNetwork        │  1.2 K │
│ 4  │ encoder_vsn                       │ _VariableSelectionNetwork        │  3.7 K │
│ 5  │ decoder_vsn                       │ _VariableSelectionNetwork        │  1.2 K │
│ 6  │ static_context_grn                │ _GatedResidualNetwork            │  1.1 K │
│ 7  │ static_context_hidden_encoder_grn │ _GatedResidualNetwork            │  1.1 K │
│ 8  │ static_context_cell_encoder_grn   │ _GatedResidualNetwork            │  1.1 K │
│ 9  │ static_context_enrichment         │ _GatedResidualNetwork            │  1.1 K │
│ 10 │ lstm_encoder                      │ LSTM                             │  4.4 K │
│ 11 │ lstm_decoder                      │ LSTM                             │  4.4 K │
│ 12 │ post_lstm_gan                     │ _GateAddNorm                     │    576 │
│ 13 │ static_enrichment_grn             │ _GatedResidualNetwork            │  1.4 K │
│ 14 │ multihead_attn                    │ _InterpretableMultiHeadAttention │    676 │
│ 15 │ post_attn_gan                     │ _GateAddNorm                     │    576 │
│ 16 │ feed_forward_block                │ _GatedResidualNetwork            │  1.1 K │
│ 17 │ pre_output_gan                    │ _GateAddNorm                     │    576 │
│ 18 │ output_layer                      │ Linear                           │    289 │
└────┴───────────────────────────────────┴──────────────────────────────────┴────────┘

Trainable params: 24.3 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.3 K                                                                                               
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=5` reached.


In [9]:
# model_nlinear = NLinearModel.load_from_checkpoint(name, best=True)
model_tft = TFTModel.load_from_checkpoint(name, best=True, map_location=torch.device("cuda:0"))

In [10]:
model_tft.model_params

OrderedDict([('hidden_size', 16),
             ('lstm_layers', 2),
             ('num_attention_heads', 4),
             ('full_attention', False),
             ('feed_forward', 'GatedResidualNetwork'),
             ('dropout', 0.25),
             ('hidden_continuous_size', 16),
             ('categorical_embedding_sizes', None),
             ('add_relative_index', False),
             ('loss_fn', None),
             ('likelihood', None),
             ('norm_type', 'LayerNorm'),
             ('input_chunk_length', 15),
             ('output_chunk_length', 5),
             ('torch_metrics', MeanAbsoluteError()),
             ('optimizer_cls', torch.optim.adam.Adam),
             ('optimizer_kwargs', None),
             ('lr_scheduler_cls', None),
             ('lr_scheduler_kwargs', None),
             ('batch_size', 32),
             ('n_epochs', 5),
             ('model_name', 'model_tft(volu_only)'),
             ('log_tensorboard', True),
             ('nr_epochs_val_period', 1),
  

In [ ]:
# model_nlinear.predict(
#     input_chunk_length=INPUT_CHUNK,
#     output_chunk_length=OUTPUT_CHUNK,
#     past_covarites=past_test_scaled
# )

In [11]:
hist_emb = historical_predictions(
    model_tft, target_val_scaled, INPUT_CHUNK, OUTPUT_CHUNK, RETRAIN, LAST, \
        covariates=True, past=past_val_scaled, future=future_val
    )

model_tft(volu_only)


  0%|          | 0/47 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/andrei/miniconda3/envs/hse/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /mnt/windows/wd_blue/HSE_Masters/darts_logs/model_tft(volu_only)/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at /mnt/windows/wd_blue/HSE_Masters/darts_logs/model_tft(volu_only)/checkpoints/best-epoch=4-val_loss=0.63.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                              ┃ Type                             ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ criterion                         │ MSELoss                          │      0 │
│ 1  │ train_metrics                     │ MetricCollection                 │      0 │
│ 2  │ val_metrics                       │ MetricCollection                 │      0 │
│ 3  │ input_embeddings                  │ _MultiEmbedding                  │      0 │
│ 4  │ static_covariates_vsn             │ _VariableSelectionNetwork        │  1.2 K │
│ 5  │ encoder_vsn                       │ _VariableSelectionNetwork        │  3.7 K │
│ 6  │ decoder_vsn                       │ _VariableSelectionNetwork        │  1.2 K │
│ 7  │ static_context_grn                │ _GatedResidualNetwork            │  1.1 K │
│ 8  │ static_context_hidden_encoder_grn │ _GatedResidualNetwork            │  1.1 K │
│ 9  │ static_context_cell_encoder_grn   │ _GatedResidualNetwork            │  1.1 K │
│ 10 │ static_context_enrichment         │ _GatedResidualNetwork            │  1.1 K │
│ 11 │ lstm_encoder                      │ LSTM                             │  4.4 K │
│ 12 │ lstm_decoder                      │ LSTM                             │  4.4 K │
│ 13 │ post_lstm_gan                     │ _GateAddNorm                     │    576 │
│ 14 │ static_enrichment_grn             │ _GatedResidualNetwork            │  1.4 K │
│ 15 │ multihead_attn                    │ _InterpretableMultiHeadAttention │    676 │
│ 16 │ post_attn_gan                     │ _GateAddNorm                     │    576 │
│ 17 │ feed_forward_block                │ _GatedResidualNetwork            │  1.1 K │
│ 18 │ pre_output_gan                    │ _GateAddNorm                     │    576 │
│ 19 │ output_layer                      │ Linear                           │    289 │
└────┴───────────────────────────────────┴──────────────────────────────────┴────────┘

Trainable params: 24.3 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.3 K                                                                                               
Total estimated model params size (MB): 0

...

┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                              ┃ Type                             ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ criterion                         │ MSELoss                          │      0 │
│ 1  │ train_metrics                     │ MetricCollection                 │      0 │
│ 2  │ val_metrics                       │ MetricCollection                 │      0 │
│ 3  │ input_embeddings                  │ _MultiEmbedding                  │      0 │
│ 4  │ static_covariates_vsn             │ _VariableSelectionNetwork        │  1.2 K │
│ 5  │ encoder_vsn                       │ _VariableSelectionNetwork        │  3.7 K │
│ 6  │ decoder_vsn                       │ _VariableSelectionNetwork        │  1.2 K │
│ 7  │ static_context_grn                │ _GatedResidualNetwork            │  1.1 K │
│ 8  │ static_context_hidden_encoder_grn │ _GatedResidualNetwork            │  1.1 K │
│ 9  │ static_context_cell_encoder_grn   │ _GatedResidualNetwork            │  1.1 K │
│ 10 │ static_context_enrichment         │ _GatedResidualNetwork            │  1.1 K │
│ 11 │ lstm_encoder                      │ LSTM                             │  4.4 K │
│ 12 │ lstm_decoder                      │ LSTM                             │  4.4 K │
│ 13 │ post_lstm_gan                     │ _GateAddNorm                     │    576 │
│ 14 │ static_enrichment_grn             │ _GatedResidualNetwork            │  1.4 K │
│ 15 │ multihead_attn                    │ _InterpretableMultiHeadAttention │    676 │
│ 16 │ post_attn_gan                     │ _GateAddNorm                     │    576 │
│ 17 │ feed_forward_block                │ _GatedResidualNetwork            │  1.1 K │
│ 18 │ pre_output_gan                    │ _GateAddNorm                     │    576 │
│ 19 │ output_layer                      │ Linear                           │    289 │
└────┴───────────────────────────────────┴──────────────────────────────────┴────────┘

Trainable params: 24.3 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.3 K                                                                                               
Total estimated model params size (MB): 0

/home/andrei/miniconda3/envs/hse/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Output()

RuntimeError: Inference tensors cannot be saved for backward. To work around you can make a clone to get a normal tensor and use it in autograd.

In [ ]:
def diplay_prediction(hist, target, inverse=False, scaler=None):
    if inverse:
        hist, target = inverse_func(scaler, hist, target)

    hist.plot(label='predict')
    target[0][30:].plot(label='true')

    error_print(target[0], hist)

    return hist

In [ ]:
hist = diplay_prediction(hist_emb, target_val_scaled, inverse=True, scaler=scaler_target)
print(name)